In [67]:
import tensorflow as tf

def hard_sigmoid(x):
    return tf.nn.relu6(x + 3) / 6

def hard_swish(x):
    return x * hard_sigmoid(x)

def conv_bn_activation(x, filters, kernel_size, strides=1, padding='same', activation=None):
    x = tf.keras.layers.Conv3D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if activation:
        x = tf.keras.layers.Activation(hard_swish)(x)
    return x

def bottleneck(x, filters, kernel_size, strides=1, expansion=4, activation=None):
    channel_axis = 1 if tf.keras.backend.image_data_format() == 'channels_first' else -1
    input_shape = tf.keras.backend.int_shape(x)
    input_filters = input_shape[channel_axis]
    t = tf.keras.layers.Conv3D(input_filters * expansion, 1, padding='same')(x)
    t = tf.keras.layers.BatchNormalization()(t)
    if activation == 'hard_sigmoid':
        t = tf.keras.layers.Activation(hard_sigmoid)(t)
    else:
        t = tf.keras.layers.Activation(hard_swish)(t)
    t = conv_bn_activation(t, filters, kernel_size, strides)
    t = tf.keras.layers.BatchNormalization()(t)
    if input_filters == filters and strides == 1:
        shortcut = x
    else:
        shortcut = conv_bn_activation(x, filters, 1, strides)
    output = tf.keras.layers.add([shortcut, t])
    return output

def MobileNetV3(input_shape=(16, 224, 224, 3), alpha=1.0, classes=5):
    img_input = tf.keras.layers.Input(shape=input_shape)

    # Stem
    x = conv_bn_activation(img_input, int(16 * alpha), 3, strides=2, activation='hard_swish')

    # Blocks
    x = bottleneck(x, int(16 * alpha), 3, expansion=1, activation='hard_sigmoid')
    x = bottleneck(x, int(24 * alpha), 3, strides=2, expansion=4, activation='hard_sigmoid')
    x = bottleneck(x, int(24 * alpha), 3, expansion=3, activation='hard_sigmoid')
    x = bottleneck(x, int(40 * alpha), 5, strides=2, expansion=3, activation='hard_sigmoid')
    x = bottleneck(x, int(40 * alpha), 5, expansion=3, activation='hard_sigmoid')
    x = bottleneck(x, int(40 * alpha), 5, expansion=3, activation='hard_sigmoid')
    x = bottleneck(x, int(80 * alpha), 3, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(80 * alpha), 3, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(80 * alpha), 3, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(80 * alpha), 3, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(112 * alpha), 5, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(112 * alpha), 5, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(160 * alpha), 5, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(160 * alpha), 5, expansion=6, activation='hard_swish')
    x = bottleneck(x, int(160 * alpha), 5, expansion=6, activation='hard_swish')

    # Head
    x = conv_bn_activation(x, int(960 * alpha),1, padding='same', activation='hard_swish')
    x = tf.keras.layers.GlobalAveragePooling3D()(x)
    x = tf.keras.layers.Reshape((1,1, 1, int(960 * alpha)))(x)
    x = tf.keras.layers.Conv3D(int(1280 * alpha), 1, padding='same')(x)
    x = tf.keras.layers.Activation(hard_swish)(x)
    x = tf.keras.layers.Conv1D(classes, 1, padding='same', activation='softmax')(x)
    x = tf.keras.layers.Reshape((classes,))(x)
    
    model = tf.keras.models.Model(img_input, x, name='mobilenetv3')

    return model

In [56]:
from keras.models import *

In [68]:
model = MobileNetV3()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = [tf.keras.metrics.CategoricalAccuracy()]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [61]:
model

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'reshape_17')>